In [1]:
# Importing recommended libraries
import cv2  
import os
from matplotlib import pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import ResNet50

In [2]:
# Main directory where model data is saved
mainDataDirectory = "C:/Users/seyed/Music/Brain Tumor - Model Data/"

In [3]:
# Categories of image classification
imageCategories = ["No", "Yes"]

# Directories for training and testing data
trainDirectory = os.path.join(mainDataDirectory, 'Train')
testDirectory = os.path.join(mainDataDirectory, 'Test')

In [4]:
# Function to read images from a directory and add labels to each image
def read_images_from_directory(directory):
    images = []
    labels = []
    for imageCategory in imageCategories:
        category_path = os.path.join(directory, imageCategory)
        label = imageCategories.index(imageCategory)
        
        for filename in os.listdir(category_path):
            image_path = os.path.join(category_path, filename)
            image = cv2.imread(image_path)
            if image is not None:
                images.append(image)
                labels.append(label)
            else:
                print(f"Error in reading the image: {image_path}")
    return images, labels

In [5]:
# Read training images
train_images, train_labels = read_images_from_directory(trainDirectory)

# Read testing images
test_images, test_labels = read_images_from_directory(testDirectory)

# Convert the lists to numpy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [6]:
# Encoding the label
label_encoder = LabelEncoder()
train_label_encoded = label_encoder.fit_transform(train_labels)
test_label_encoded = label_encoder.fit_transform(test_labels)

# Convert the encoded labels to one-hot encoding
train_labels_onehot = to_categorical(train_label_encoded, num_classes=2)
test_labels_onehot = to_categorical(test_label_encoded, num_classes=2)

In [7]:
# Running the tf in cpu
with tf.device('/cpu:0'):
    # Loading the vgg16 model and setting up the image size to train
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

    # Freeze the layers
    for layer in base_model.layers:
        layer.trainable = False
    
    # Update the model architecture
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(2, activation='softmax'))  

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model
    history = model.fit(train_images, train_labels_onehot, epochs=10, batch_size=16, validation_data=(test_images, test_labels_onehot))


94765736/94765736 [==============================] - 15s 0us/step
Epoch 1/10
2595/2595 [==============================] - 1553s 597ms/step - loss: 0.5464 - accuracy: 0.9417 - val_loss: 0.0445 - val_accuracy: 0.9832
Epoch 2/10
2595/2595 [==============================] - 1547s 596ms/step - loss: 0.1280 - accuracy: 0.9549 - val_loss: 0.0675 - val_accuracy: 0.9796
Epoch 3/10
2595/2595 [==============================] - 1546s 596ms/step - loss: 0.1203 - accuracy: 0.9537 - val_loss: 0.0566 - val_accuracy: 0.9765
Epoch 4/10
2595/2595 [==============================] - 1546s 596ms/step - loss: 0.1125 - accuracy: 0.9612 - val_loss: 0.0643 - val_accuracy: 0.9855
Epoch 5/10
2595/2595 [==============================] - 1545s 595ms/step - loss: 0.0874 - accuracy: 0.9710 - val_loss: 0.0235 - val_accuracy: 0.9907
Epoch 6/10
2595/2595 [==============================] - 1544s 595ms/step - loss: 0.0785 - accuracy: 0.9755 - val_loss: 0.0290 - val_accuracy: 0.9903
Epoch 7/10
2595/2595 [==================

In [8]:
# Save the model
model.save('C:/Users/seyed/Music/Brain Tumor - Model/TumorDetectionModel_ResNet50.h5')

In [9]:
# Displayed the trained model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                                 
 dense (Dense)               (None, 256)               33554688  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 514       
                                                                 
Total params: 57,142,914
Trainable params: 33,555,202
Non-trainable params: 23,587,712
_________________________________________________________________
